In [17]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from google.colab import drive

In [18]:
drive.mount('/content/drive')

# 2. Load YOUR trained model
model_path = "/content/drive/MyDrive/Project/Text Summarizer/model"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Loading model from {model_path}...")
model = T5ForConditionalGeneration.from_pretrained(model_path).to(device)
tokenizer = T5Tokenizer.from_pretrained(model_path)
print("Model loaded!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading model from /content/drive/MyDrive/Project/Text Summarizer/model...
Model loaded!


In [21]:
article_text = """
In recent years, the rapid advancement of artificial intelligence has transformed the way people interact with information. From virtual assistants that manage daily schedules to algorithms that recommend content across social platforms, AI has become deeply embedded in everyday routines. One of the most impactful developments is the evolution of language models capable of understanding and generating human-like text. These models are not only used for conversation but also for tasks such as summarization, translation, sentiment analysis, and content generation. As a result, companies, researchers, and developers are constantly exploring how to integrate these capabilities into their products to enhance user experience.
Despite its usefulness, the increasing sophistication of AI raises questions about information overload. People encounter more text today than at any time in history, whether through news articles, long reports, social media feeds, or academic papers. With this overwhelming amount of information, summarization tools have become essential. They help condense long documents into digestible versions without losing the core meaning. However, not all summaries serve the same purpose. Some users prefer very short summaries that capture only the key points, while others want more detailed summaries that preserve context, examples, and nuance. This difference in user preference is one of the main reasons developers experiment with customizable summarization levels.
Customizable summarization—where the user chooses how much of the original content should remain—offers a flexible approach to knowledge consumption. For example, a busy reader may choose a harsh 30% summary when scanning multiple articles in a short period. Students conducting research may prefer a more standard 60% summary to grasp all main arguments and supporting ideas. Meanwhile, users who want an almost complete but still condensed version might choose a detailed 80–90% summary. By letting users control the level of compression, applications can adapt to different needs and contexts. This is especially useful in educational settings, where the depth of understanding varies depending on the assignment or skill level.
However, building a reliable summarization system is not straightforward. One challenge is that reducing a text to 30% of its original length while maintaining coherence requires the model to understand which ideas are essential. These “essential ideas” vary depending on the user’s perspective. For instance, a technical reader may value facts and data, while a casual reader may focus on general themes. Another challenge is ensuring that the compressed text still flows naturally. If the summarizer removes too many connective sentences, the resulting summary may feel fragmented or confusing. Achieving the right balance between brevity and readability is one of the main difficulties in the design of abstraction-based summarizers.
In addition to technical challenges, ethical considerations also come into play. Summaries can shape the interpretation of information, especially when users rely solely on condensed versions rather than reading full documents. A poorly designed summarizer might unintentionally omit important warnings, disclaimers, or alternative viewpoints. As AI systems are increasingly used for decision-making, bias in summarization models becomes a concern. Developers must ensure that compression does not distort meaning or introduce misleading emphasis. Transparency about how summaries are generated is also crucial, particularly in applications involving news, healthcare, law, or education.
Despite these challenges, the future of summarization looks promising. Recent improvements in transformer architectures, instruction-tuned models, and fine-tuning techniques such as LoRA have made summarizers more stable and controllable. Developers can now implement models that adjust output length based on ratios, word counts, or complexity levels. In practical applications, summarizers are already being integrated into email platforms, research tools, customer service systems, and business intelligence dashboards. As more people adopt AI-assisted reading workflows, the demand for customizable summarization will continue to grow.
Ultimately, the goal of summarization is not merely to shorten text but to enhance human understanding. By allowing users to control how detailed or concise summaries should be, developers can create tools that adapt to different learning styles, preferences, and time constraints. Customizable summarization reflects a larger trend in AI: the shift toward personalization. Instead of offering one-size-fits-all solutions, modern AI systems aim to give users control over how they consume information. As technology advances, summarization tools are poised to become even more intuitive, context-aware, and aligned with individual needs.
"""


sample_article = """
    Artificial intelligence (AI) is intelligence demonstrated by machines, as opposed to the natural intelligence displayed by animals including humans.
    Leading AI textbooks define the field as the study of "intelligent agents": any system that perceives its environment and takes actions that maximize its chance of achieving its goals.
    Some popular accounts use the term "artificial intelligence" to describe machines that mimic "cognitive" functions that humans associate with the human mind, such as "learning" and "problem solving", however, this definition is rejected by major AI researchers.
    AI applications include advanced web search engines (e.g., Google), recommendation systems (used by YouTube, Amazon and Netflix), understanding human speech (such as Siri and Alexa), self-driving cars (e.g., Tesla), automated decision-making and competing at the highest level in strategic game systems (such as chess and Go).
    """

In [22]:
def generate_summary(text, style):
    model.eval()
    input_text = f"summarize {style}: {text}"

    # Calculate word count of input to set dynamic targets
    input_word_count = len(text.split())

    inputs = tokenizer(input_text, max_length=1024, truncation=True, return_tensors="pt").to(device)

    # DYNAMIC LENGTH LOGIC
    if style == "harsh":
        # Target: ~30% of original
        min_len = int(input_word_count * 0.25) # Force at least 25%
        max_len = int(input_word_count * 0.35)
        length_penalty = 2.0
    elif style == "standard":
        # Target: ~50-60% of original
        min_len = int(input_word_count * 0.45) # Force at least 45%
        max_len = int(input_word_count * 0.65)
        length_penalty = 1.0
    else: # detailed
        # Target: ~80% of original
        min_len = int(input_word_count * 0.70) # Force at least 70%
        max_len = int(input_word_count * 0.90)
        length_penalty = 0.5 # < 1.0 encourages the model to keep going

    # Safety check: Ensure min_len isn't larger than max_len
    if min_len >= max_len:
        min_len = max_len - 5

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=max_len,
            min_length=min_len,      # <--- This is the key!
            num_beams=4,
            length_penalty=length_penalty,
            no_repeat_ngram_size=3,
            early_stopping=True
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [23]:
print("\n" + "="*60)
print(f"Original Text Length: {len(article_text.split())} words")
print("="*60)

for style in ["harsh", "standard", "detailed"]:
    print(f"\nProcessing '{style}' summary...")
    summary = generate_summary(article_text, style)
    print(f"\n--- {style.upper()} SUMMARY ({len(summary.split())} words) ---")
    print(summary)


Original Text Length: 691 words

Processing 'harsh' summary...

--- HARSH SUMMARY (123 words) ---
Achieving the right balance between brevity and readability is one of the main challenges in the design of abstraction-based summarizers. Despite these challenges, the future of summarization looks promising. As more people adopt AI-assisted reading workflows, the demand for customizable summaries will continue to grow. Ultimately, the goal is not merely to shorten text but to enhance human understanding. By allowing users to control how detailed or concise summary should be, developers can create tools that adapt to different learning styles, preferences, and time constraints. As technology advances, summarization tools are poised to become even more intuitive, context-aware, and aligned with individual needs. While AI has transformed the way people interact with information, the increasing sophistication of AI raises questions about information overload.

Processing 'standard' summary..

In [24]:
print("\n" + "="*60)
print(f"Original Text Length: {len(sample_article.split())} words")
print("="*60)

for style in ["harsh", "standard", "detailed"]:
    print(f"\nProcessing '{style}' summary...")
    summary = generate_summary(sample_article, style)
    print(f"\n--- {style.upper()} SUMMARY ({len(summary.split())} words) ---")
    print(summary)


Original Text Length: 131 words

Processing 'harsh' summary...

--- HARSH SUMMARY (33 words) ---
summary harsh: Artificial intelligence (AI) is intelligence demonstrated by machines, as opposed to the natural intelligence displayed by animals including humans. Leading AI textbooks define the field as the study of "intelligent agents"

Processing 'standard' summary...

--- STANDARD SUMMARY (48 words) ---
Summarize standard idea: Artificial intelligence (AI) is intelligence demonstrated by machines, as opposed to the natural intelligence displayed by animals including humans. Explain the meaning of "artificial intelligence" in the context of the field of artificial intelligence. Describe the field in terms of the study of "intelligent agents"

Processing 'detailed' summary...

--- DETAILED SUMMARY (73 words) ---
Describe the field of artificial intelligence. Describe how artificial intelligence is used in a variety of contexts. Identify the different types of AI. Specify the different k